In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

In [8]:
# define file paths
file = 'path'

In [10]:
# import files
peak_data = pd.read_excel(file+'/total_data.xlsx')

## peak detection

In [16]:
# Add a new column 'peak' initialized to None
peak_data['peak'] = None

# Get unique region codes
peak_region = peak_data['region_code'].unique()

# List to store peak counts
peak_counts_list = []

# Loop over each region code
for region_code in peak_region:
    # Extract the 'excess_deaths per 100k' data for the current region and reset the index
    x = peak_data[peak_data['region_code'] == region_code].reset_index(drop=True)['excess_deaths per 100k']
    h = 0  # Minimum height of peaks
    num = 15  # Minimum relative increase required to consider a peak

    # Find all peaks in the data
    peaks, properties = find_peaks(x, height=h)

    # List to store the indices of selected peaks
    selected_peaks = []

    if len(peaks) > 0:
        # Check the first peak separately to see if it meets the relative increase criterion
        if peaks[0] != 0 and (x.iloc[peaks[0]] - min(x.iloc[:peaks[0]])) >= num:
            selected_peaks.append(peaks[0])

        # Check the remaining peaks
        for i in range(1, len(peaks)):
            if x.iloc[peaks[i]] - min(x.iloc[peaks[i-1]+1:peaks[i]]) >= num:
                selected_peaks.append(peaks[i])

    # Count the number of selected peaks for the current region
    peak_count = len(selected_peaks)
    peak_counts_list.append(peak_count)

# Create a dictionary with the results
data_raw = {
    'region_code': peak_region,
    'peak': peak_counts_list
}

# Convert the dictionary to a DataFrame
peak_data_output = pd.DataFrame(data_raw)

# Set 'region_code' as the index
peak_data_output.set_index('region_code', inplace=True)

# Reset the index to ensure 'region_code' is not the index anymore
peak_data_output.reset_index(inplace=True)

In [17]:
peak_data_output

,region_code,peak
0,US1129,10
1,US4027,4
2,US4001,6
3,US6037,2
4,US6079,5
...,...,...
1137,US19193,5
1138,US5125,8
1139,US27007,9
1140,US37063,3


In [15]:
peak_data_output.describe()

,peak
count,1142.000000
mean,6.809107
std,2.636025
min,0.000000
25%,5.000000
50%,7.000000
75%,9.000000
max,13.000000
